In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PyQt4.QtGui import QImage

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def loadCameraCalibration():
    """
    Load camera intrinsic matrix from file
    """
    return np.loadtxt("./util/intrinsic_mat.cfg")

In [ ]:
camera_calibration = loadCameraCalibration()
camera_calibration

# Load the RGB and Depth Images & Show Raw Imgs

In [ ]:
rgb_img = cv2.imread('./rgb_7.png')
depth_img = cv2.imread('./depth_7.png')
depth_data = np.load('./depth_frame_data_7.npy')

In [ ]:
rgb_img.shape

In [ ]:
plt.imshow(rgb_img[...,::-1])

In [ ]:
plt.imshow(depth_img[...,::-1])

In [ ]:
def convertDepthFrame(depthFrameData):
    """ Converts frame to a colormaped format suitable for Qt  
        Note: this cycles the spectrum over the lowest 8 bits
    """
    try:

        """ 
        Convert Depth frame to rudimentary colormap
        """
        DepthHSV = np.zeros((480,640,3)).astype(np.uint8)
        DepthCM=np.array([])
        block_contours = np.array([])
        DepthHSV[...,0] = depthFrameData
        DepthHSV[...,1] = 0x9F
        DepthHSV[...,2] = 0xFF
        DepthCM = cv2.cvtColor(DepthHSV,cv2.COLOR_HSV2RGB)
        cv2.drawContours(DepthCM,block_contours,-1,(0,0,0),3)
#        img = QImage(DepthCM,
#                         DepthCM.shape[1],
#                         DepthCM.shape[0],
#                         QImage.Format_RGB888
#                         )
        return DepthCM
    except:
        return None

In [ ]:
depthImgFromData = convertDepthFrame(depth_data)

In [ ]:
plt.imshow(depthImgFromData)

# Create and mark correspondance points

In [ ]:
xs_rgb=[105,85,430,440,255]
ys_rgb=[447,100,85,425,280]


rgb_img2 = rgb_img.copy()
for i in range(len(xs_rgb)):
    rgb_img2[ys_rgb[i]-5:ys_rgb[i]+5,xs_rgb[i]-5:xs_rgb[i]+5] = [0,0,255]

In [ ]:
plt.imshow(rgb_img2[...,::-1])

In [ ]:
xs_depth=[112,80,460,480,275]
ys_depth=[460,95,65,437,280]


depth_img2 = depth_img.copy()
for i in range(len(xs_depth)):
    depth_img2[ys_depth[i]-5:ys_depth[i]+5,xs_depth[i]-5:xs_depth[i]+5] = [0,0,255]

In [ ]:
plt.imshow(depth_img2[...,::-1])

# Determine the Affine Transformation Between the Two

In [ ]:
def getAffineTransformNonCV(coord1, coord2):
        """
        Given 2 sets of corresponding coordinates, 
        find the affine matrix transform between them.

        TODO: Rewrite this function to take in an arbitrary number of coordinates and 
        find the transform without using cv2 functions
        """
        pts1 = coord1[::].astype(np.float32)
        pts2 = coord2[::].astype(np.float32)
        numPoints = len(pts1)
        A = np.zeros((numPoints*2,6))
        B = np.zeros((numPoints*2,1))
        for i in range(numPoints):
            Arow_1 = [pts1[i][0], pts1[i][1],1,0,0,0]
            Arow_2 = [0,0,0,pts1[i][0], pts1[i][1],1]
            B[i*2] = pts2[i][0]
            B[(i*2)+1] = pts2[i][1]
            A[i*2] = Arow_1
            A[(i*2)+1] = Arow_2

        AtA = np.matmul(A.T, A)
        AtA_inv = np.linalg.inv(AtA)
        pseudo_invA = np.matmul(AtA_inv,A.T)
        result = np.matmul(pseudo_invA,B)
        return [[result[0][0],result[1][0],result[2][0]],[result[3][0],result[4][0],result[5][0]],[0,0,1]]

In [ ]:
xys_rgb = np.array([(a,b) for a, b in zip(*(xs_rgb, ys_rgb))]) 
xys_depth = np.array([(a,b) for a, b in zip(*(xs_depth, ys_depth))]) 

In [ ]:
rgb_to_depth_affine = getAffineTransformNonCV(xys_depth, xys_rgb)

In [ ]:
rgb_to_depth_affine

### Quick verification of Affine generation function vs opencv

In [ ]:
xys_rgb2 = np.array([(a,b) for a, b in zip(*(xs_rgb[0:3], ys_rgb[0:3]))],np.float32) 
xys_depth2 = np.array([(a,b) for a, b in zip(*(xs_depth[0:3], ys_depth[0:3]))],np.float32) 

In [ ]:
rgb_to_depth_affine_cv2 = cv2.getAffineTransform( xys_depth2[0:3], xys_rgb2[0:3])

In [ ]:
rgb_to_depth_affine_cv2

# Apply the Affine Transformation and check work with previous points

In [ ]:
def applyAffine(frame, affineMatrix):
        input_shape = frame.shape
        result = np.zeros(input_shape, dtype=np.uint64)
        for input_y in range(input_shape[0]):
            for input_x in range(input_shape[1]):
                source_mat = np.array([float(input_x),float(input_y),1.0]).T
                dest_mat = np.matmul(affineMatrix,source_mat)
                dest_x = int(dest_mat[0])
                dest_y = int(dest_mat[1])
                if(dest_x >= 0 and dest_x < input_shape[1] and dest_y >= 0 and dest_y < input_shape[0]):
                    result[dest_y][dest_x] = frame[input_y][input_x]
        return result

In [ ]:
depth_transformed = applyAffine(depth_img, rgb_to_depth_affine)

depth_transformed2 = depth_transformed.copy()
for i in range(len(xs_rgb)):
    depth_transformed2[ys_rgb[i]-5:ys_rgb[i]+5,xs_rgb[i]-5:xs_rgb[i]+5] = [0,0,255]
    
depthImgFromData = convertDepthFrame(depth_transformed2)


In [ ]:
plt.imshow(depth_transformed2[...,::-1])

In [ ]:
depth_data_transformed = applyAffine(depth_data, rgb_to_depth_affine)

depth_fromData_transformed2 = depth_data_transformed.copy()
depthImgFromData2 = convertDepthFrame(depth_fromData_transformed2)
for i in range(len(xs_rgb)):
    depthImgFromData2[ys_rgb[i]-5:ys_rgb[i]+5,xs_rgb[i]-5:xs_rgb[i]+5] = [0,0,255]

plt.imshow(depthImgFromData2)

### Quick check against CV2 results

In [ ]:
depth_transformed_cv2 = cv2.warpAffine(depth_img,rgb_to_depth_affine_cv2,(depth_img.shape[1], depth_img.shape[0]))

depth_transformed2 = depth_transformed_cv2.copy()
for i in range(len(xs_rgb)):
    depth_transformed2[ys_rgb[i]-5:ys_rgb[i]+5,xs_rgb[i]-5:xs_rgb[i]+5] = [0,0,255]

In [ ]:
plt.imshow(depth_transformed2[...,::-1])

# Get the transform from the model to the camera

In [ ]:
def registerExtrinsicMatrix(rVec, tVec):
    extrinsicTranslation = tVec
    extrinsicRotation = rVec
    extrinsicRotation_matrix = cv2.Rodrigues(rVec)[0]
    cameraExtrinsic = np.zeros((4,4))
    cameraExtrinsic[0][3] = tVec[0]
    cameraExtrinsic[1][3] = tVec[1]
    cameraExtrinsic[2][3] = tVec[2]
    cameraExtrinsic[3][3] = 1.0
    for i in [0,1,2]:
        for j in [0,1,2]:
            cameraExtrinsic[i][j] = extrinsicRotation_matrix[i][j]
    return cameraExtrinsic

In [ ]:
def imgXyToCamXYZ(cameraCalibration, imgX, imgY, depth_data):
    u0 = cameraCalibration[0][2]
    v0 = cameraCalibration[1][2]
    alpha = cameraCalibration[0][0]
    beta = cameraCalibration[1][1]
    z_pixel = depth_data[imgY][imgX]
    Zc = 123.6 * np.tan(z_pixel/2842.5 + 1.1863)
    Xc = ((imgX - u0) * Zc)/alpha
    Yc = ((imgY - v0) * Zc)/beta
    return np.array([Xc,Yc,Zc])

In [ ]:
model_points = np.array([\
            [-305.0,-305.0,0.0],\
                [-305.0,305.0,0.0],\
                    [305.0,305.0,0.0],\
                        [305.0,-305.0,0.0],\
                            [0.0,0.0,128.75]])

In [ ]:
image_points = np.float32(xys_rgb)

In [ ]:
(success, rot_vec, trans_vec) = cv2.solvePnP(model_points, \
                                    image_points,\
                                    camera_calibration,\
                                    None)

In [ ]:
camera_world_transform = registerExtrinsicMatrix(rot_vec, trans_vec)
camera_world_transform

In [ ]:
points=[[430,120]]

In [ ]:
for [y,x] in points:
    print(imgXyToCamXYZ(camera_world_transform, camera_calibration, x, y ,depth_data_transformed))

In [ ]:
depth_fromData_transformed2 = depth_data_transformed.copy()
depthImgFromData2 = convertDepthFrame(depth_fromData_transformed2)
for [y,x] in points:
    depthImgFromData2[y-5:y+5,x-5:x+5] = [0,0,255]
plt.imshow(depthImgFromData2)